THIS IS AN EXPERIMENTAL FILE AND NOT PART OF SUBMISSION!!! IT HAS FUNCTIONAL AND NON FUNCTIONAL EXPEREMENTAL VERSION OF INFERENC SCRIPTS

In [ ]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model

# ── 1) Parse TestOptions via sys.argv ─────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot',   './my_dataset',
    '--name',       'human2arcane',
    '--model',      'cycle_gan',
    '--phase',      'test',
    '--num_test',   '1',
    '--load_size',  '256',
    '--crop_size',  '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Build full-frame preprocess transform (match test.py) ─────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to model input
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Start capture ──────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to PIL and preprocess
    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    input_tensor = preprocess(img_pil).unsqueeze(0).to(model.device)

    # Stylize full frame
    with torch.no_grad():
        fake = model.netG_A(input_tensor)

    # Convert back to uint8 image
    fake_np = fake.squeeze().cpu().permute(1, 2, 0).numpy()  # HxWxC in [-1,1]
    fake_img = ((fake_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)

    # Resize stylized image back to original frame size
    fake_bgr = cv2.cvtColor(fake_img, cv2.COLOR_RGB2BGR)
    output = cv2.resize(fake_bgr, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_LINEAR)

    # Show result
    cv2.imshow('Human 2 Arcane - Full Frame', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./my_dataset                  	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [ ]:
#this is for webcam

import cv2
import numpy as np
import sys
import torch # For torch.no_grad()
from PIL import Image, ImageEnhance
import torchvision.transforms as transforms
# Ensure these custom modules are in your Python path or the same directory
# from options.test_options import TestOptions
# from models import create_model
import pyvirtualcam

# --- Fallback Mock Model (if your CycleGAN classes aren't found) ---
try:
    from options.test_options import TestOptions
    from models import create_model
    MODEL_CLASSES_AVAILABLE = True
except ImportError:
    print("WARNING: 'options.test_options' or 'models.create_model' not found.")
    print("Using a MOCK model. Output will not be style-transferred.")
    MODEL_CLASSES_AVAILABLE = False

    class MockTestOptions:
        def __init__(self):
            self.gpu_ids = []
            self.isTrain = False
            self.dataroot = './my_dataset'
            self.name = 'human2arcane_mock'
            self.model = 'cycle_gan'
            self.phase = 'test'
            self.num_test = 1
            self.load_size = 256
            self.crop_size = 256
            self.no_flip = True
            self.direction = 'AtoB'
            self.dataset_mode = 'unaligned'
            self.serial_batches = True
            self.num_threads = 0
            self.batch_size = 1
            self.max_dataset_size = float("inf")
            self.input_nc = 3
            self.output_nc = 3
            self.ngf = 64
            self.ndf = 64
            self.netG = 'resnet_9blocks'
            self.netD = 'basic'
            self.norm = 'instance'
            self.init_type = 'normal'
            self.init_gain = 0.02
            self.no_dropout = True
            self.verbose = False
            self.preprocess = 'resize_and_crop'

        def parse(self):
            print("MockTestOptions parsed.")
            self.gpu_ids = [] if not torch.cuda.is_available() else [0]
            return self

    class MockModel:
        def __init__(self, opt):
            self.opt = opt
            self.device = torch.device('cuda:{}'.format(opt.gpu_ids[0])) if opt.gpu_ids and torch.cuda.is_available() else torch.device('cpu')
            print(f"MockModel using device: {self.device}")
            self.netG_A = lambda x: torch.clamp(x * 0.8, -1.0, 1.0)
            self.netG = self.netG_A

        def setup(self, opt):
            print(f"MockModel setup with options: {opt.name}")
        def eval(self):
            print("MockModel set to eval mode.")
        def set_input(self, input_data):
            self.real_A = input_data['A'].to(self.device)
            # Mock doesn't strictly need B for test, but real model might
            if 'B' in input_data:
                self.real_B = input_data['B'].to(self.device)
            self.image_paths = input_data.get('A_paths', [])
        def test(self):
            with torch.no_grad():
                self.fake_B = self.netG_A(self.real_A)
        def get_current_visuals(self):
            visual_ret = {}
            if hasattr(self, 'real_A'): visual_ret['real_A'] = self.real_A
            if hasattr(self, 'fake_B'): visual_ret['fake_B'] = self.fake_B
            return visual_ret

# --- 1) Parse Options and Load Model ---
print("Initializing CycleGAN model...")
if MODEL_CLASSES_AVAILABLE:
    sys.argv = [
        'test.py',
        '--dataroot', './my_dataset',
        '--name', 'human2arcane',
        '--model', 'cycle_gan',
        '--phase', 'test',
        '--num_test', '1',
        '--load_size', '256',
        '--crop_size', '256',
        '--no_flip',
        # Ensure your command line options for testing are appropriate
        # For example, some models might expect --dataset_mode unaligned even for single image test
        # or specific --netG, --norm options if not default in your saved model.
        # Usually, the options saved with the model during training are loaded.
    ]
    opt = TestOptions().parse()
    if hasattr(opt, 'isTrain'): opt.isTrain = False
    model = create_model(opt)
    model.setup(opt)
    model.eval()
    model_device = model.device
else:
    opt = MockTestOptions().parse()
    model = MockModel(opt)
    model.setup(opt)
    model.eval()
    model_device = model.device
print(f"Model '{opt.name}' loaded. Using device: {model_device}")

# --- 2) Preprocessing for CycleGAN ---
preprocess_transform = transforms.Compose([
    transforms.Resize((opt.crop_size, opt.crop_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# --- 3) Webcam Capture ---
WEBCAM_INDEX = 0
FPS_TARGET = 30
cap = cv2.VideoCapture(WEBCAM_INDEX)
if not cap.isOpened():
    raise RuntimeError(f"❌ Could not open webcam at index {WEBCAM_INDEX}.")
cap.set(cv2.CAP_PROP_FPS, FPS_TARGET)
actual_fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Webcam actual FPS: {actual_fps if actual_fps > 0 else 'Could not get FPS, using target.'}")
ret, frame = cap.read()
if not ret:
    cap.release()
    raise RuntimeError("❌ Failed to read initial frame from webcam.")
webcam_h, webcam_w = frame.shape[:2]
print(f"Webcam opened successfully: {webcam_w}x{webcam_h}")

# --- 4) Create Separate OpenCV Windows and Trackbars ---
CONTROLS_WINDOW_NAME = "Adjustments"
PREVIEW_WINDOW_NAME = "CycleGAN Virtual Camera"
cv2.namedWindow(CONTROLS_WINDOW_NAME)
cv2.createTrackbar("Brightness", CONTROLS_WINDOW_NAME, 100, 200, lambda x: None)
cv2.createTrackbar("Contrast", CONTROLS_WINDOW_NAME, 100, 200, lambda x: None)
cv2.namedWindow(PREVIEW_WINDOW_NAME)

# --- 5) Start OBS VirtualCam ---
try:
    print(f"Attempting to initialize pyvirtualcam with: width={webcam_w}, height={webcam_h}, fps={FPS_TARGET}")
    with pyvirtualcam.Camera(width=webcam_w, height=webcam_h, fps=FPS_TARGET, backend='obs', print_fps=True) as cam:
        print(f"✅ Virtual Camera started: {cam.device} ({cam.width}x{cam.height} @ {cam.fps} FPS)")
        print("📸 Press 'q' in the preview window or Ctrl+C in console to quit.")

        while True:
            ret, frame_bgr = cap.read()
            if not ret:
                print("❌ Failed to grab frame from webcam. Exiting loop.")
                break
            brightness_val = cv2.getTrackbarPos("Brightness", CONTROLS_WINDOW_NAME)
            contrast_val = cv2.getTrackbarPos("Contrast", CONTROLS_WINDOW_NAME)
            brightness_factor = brightness_val / 100.0
            contrast_factor = contrast_val / 100.0
            frame_rgb_pil = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(frame_rgb_pil)
            enhancer_brightness = ImageEnhance.Brightness(img_pil)
            img_pil_enhanced = enhancer_brightness.enhance(brightness_factor)
            enhancer_contrast = ImageEnhance.Contrast(img_pil_enhanced)
            img_pil_final_for_gan = enhancer_contrast.enhance(contrast_factor)
            input_tensor = preprocess_transform(img_pil_final_for_gan).unsqueeze(0).to(model_device)

            with torch.no_grad():
                if MODEL_CLASSES_AVAILABLE and hasattr(model, 'set_input') and hasattr(model, 'test') and hasattr(model, 'get_current_visuals'):
                    # Standard Jun-Yan Zhu's repo structure
                    input_dict_for_set_input = {
                        'A': input_tensor,
                        'A_paths': ['dummy_A_path'], # Corresponds to input['A']
                        # Provide dummy 'B' and 'B_paths' for CycleGANModel.set_input
                        'B': input_tensor.clone().detach(), # Dummy B, content doesn't matter for netG_A(real_A)
                        'B_paths': ['dummy_B_path']  # Dummy B_paths for input['B']
                    }
                    model.set_input(input_dict_for_set_input)
                    model.test()
                    visuals = model.get_current_visuals()
                    fake_output_tensor = visuals.get('fake_B', visuals.get('fake', input_tensor))
                elif hasattr(model, 'netG_A'):
                     fake_output_tensor = model.netG_A(input_tensor)
                elif hasattr(model, 'netG'):
                     fake_output_tensor = model.netG(input_tensor)
                else:
                    print("WARNING: Could not determine model inference method. Passing through input.")
                    fake_output_tensor = input_tensor

            fake_np = fake_output_tensor.squeeze(0).cpu().permute(1, 2, 0).numpy()
            output_img_normalized = (fake_np + 1) / 2.0
            output_img_uint8 = (output_img_normalized * 255.0).clip(0, 255).astype(np.uint8)
            if output_img_uint8.ndim == 2:
                output_img_uint8 = cv2.cvtColor(output_img_uint8, cv2.COLOR_GRAY2RGB)
            elif output_img_uint8.shape[2] == 1:
                output_img_uint8 = cv2.cvtColor(output_img_uint8, cv2.COLOR_GRAY2RGB)
            if output_img_uint8.shape[0] != webcam_h or output_img_uint8.shape[1] != webcam_w:
                output_final_rgb = cv2.resize(output_img_uint8, (webcam_w, webcam_h), interpolation=cv2.INTER_LINEAR)
            else:
                output_final_rgb = output_img_uint8
            preview_display_bgr = cv2.cvtColor(output_final_rgb, cv2.COLOR_RGB2BGR)
            cv2.imshow(PREVIEW_WINDOW_NAME, preview_display_bgr)
            cv2.imshow(CONTROLS_WINDOW_NAME, frame_bgr)
            cam.send(output_final_rgb)
            cam.sleep_until_next_frame()
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("INFO: 'q' pressed, exiting.")
                break
            try:
                if cv2.getWindowProperty(PREVIEW_WINDOW_NAME, cv2.WND_PROP_VISIBLE) < 1 or \
                   cv2.getWindowProperty(CONTROLS_WINDOW_NAME, cv2.WND_PROP_VISIBLE) < 1:
                    print("INFO: A window was closed, exiting.")
                    break
            except cv2.error:
                print("INFO: OpenCV window error, likely closed. Exiting.")
                break
except KeyboardInterrupt:
    print("👋 Exiting due to KeyboardInterrupt...")
except RuntimeError as e:
    print(f"🔥 PYVIRTUALCAM RUNTIME ERROR: {e}")
    print("   Ensure OBS Studio is running (AS ADMINISTRATOR if issues persist) and the Virtual Camera is started in OBS,")
    print("   or that the chosen backend and camera parameters (resolution/FPS) are correctly configured and match OBS settings.")
except Exception as e:
    print(f"💥 An unexpected error occurred: {e}")
    import traceback
    traceback.print_exc()
finally:
    print("Releasing resources...")
    if 'cap' in locals() and cap.isOpened():
        cap.release()
    cv2.destroyAllWindows()
    print("Cleanup finished. Exiting application.")

In [ ]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions via sys.argv ─────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot',   './my_dataset',
    '--name',       'human2arcane',
    '--model',      'cycle_gan',
    '--phase',      'test',
    '--num_test',   '1',
    '--load_size',  '256',
    '--crop_size',  '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Build preprocessing transform ──────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Prepare face detector ──────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Start webcam capture ───────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcnn.detect(img_pil)
    output = frame.copy()

    if boxes is not None and len(boxes):
        h, w = frame.shape[:2]

        # Original bounding box
        x1 = int(boxes[0][0])
        y1 = int(boxes[0][1])
        x2 = int(boxes[0][2])
        y2 = int(boxes[0][3])

        # Padding percentage
        padding = 0.3  # 30% padding on each side
        w_box = x2 - x1
        h_box = y2 - y1

        # Expand bounding box by padding percentage
        x1 = max(0, x1 - int(w_box * padding))
        y1 = max(0, y1 - int(h_box * padding))
        x2 = min(w, x2 + int(w_box * padding))
        y2 = min(h, y2 + int(h_box * padding))

        # Final size after expansion
        w_box = x2 - x1
        h_box = y2 - y1

        if w_box > 0 and h_box > 0:
            # Crop & preprocess
            face_crop = img_pil.crop((x1, y1, x2, y2))
            face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)

            # Stylize
            with torch.no_grad():
                fake = model.netG_A(face_tensor)

            # Convert back to uint8 image
            fake_np = fake.squeeze().cpu().permute(1, 2, 0).numpy()  # HxWxC in [-1,1]
            fake_img = ((fake_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)

            # Resize and paste
            fake_resized = cv2.resize(fake_img, (w_box, h_box), interpolation=cv2.INTER_LINEAR)
            output[y1:y2, x1:x2] = fake_resized

    cv2.imshow('Human 2 Arcane Live', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions via sys.argv ─────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot',   './my_dataset',
    '--name',       'human2arcane',
    '--model',      'cycle_gan',
    '--phase',      'test',
    '--num_test',   '1',
    '--load_size',  '256',
    '--crop_size',  '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Image preprocessing transform ─────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Start webcam capture ──────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

# Define percentage margin around detected face
MARGIN_PCT = 0.3  # 30% expansion in all directions

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcnn.detect(img_pil)
    output = frame.copy()

    if boxes is not None and len(boxes):
        h, w = frame.shape[:2]
        x1, y1, x2, y2 = boxes[0]

        # Expand crop with margin
        box_w, box_h = x2 - x1, y2 - y1
        margin_w = box_w * MARGIN_PCT
        margin_h = box_h * MARGIN_PCT

        # Apply margin and clip to image boundaries
        x1 = int(max(0, x1 - margin_w))
        y1 = int(max(0, y1 - margin_h))
        x2 = int(min(w, x2 + margin_w))
        y2 = int(min(h, y2 + margin_h))

        # Crop, preprocess and stylize
        face_crop = img_pil.crop((x1, y1, x2, y2))
        face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)

        with torch.no_grad():
            fake = model.netG_A(face_tensor)

        # Convert to uint8 RGB image
        fake_np = fake.squeeze().cpu().permute(1, 2, 0).numpy()
        fake_img = ((fake_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)

        # Resize to match original crop and convert to BGR for OpenCV
        w_box, h_box = x2 - x1, y2 - y1
        fake_resized = cv2.resize(fake_img, (w_box, h_box), interpolation=cv2.INTER_LINEAR)
        fake_resized = cv2.cvtColor(fake_resized, cv2.COLOR_RGB2BGR)

        # Paste back into frame
        output[y1:y2, x1:x2] = fake_resized

    # Show result
    cv2.imshow('Human 2 Arcane Live', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions via sys.argv ─────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot',   './my_dataset',
    '--name',       'human2arcane',
    '--model',      'cycle_gan',
    '--phase',      'test',
    '--num_test',   '1',
    '--load_size',  '256',
    '--crop_size',  '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Image preprocessing transform ─────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Start webcam capture ──────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

# Define percentage margin around detected face
MARGIN_PCT = 0.3  # 30% expansion in all directions

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcnn.detect(img_pil)
    output = frame.copy()

    if boxes is not None and len(boxes):
        h, w = frame.shape[:2]
        x1, y1, x2, y2 = boxes[0]

        # Expand crop with margin
        box_w, box_h = x2 - x1, y2 - y1
        margin_w = box_w * MARGIN_PCT
        margin_h = box_h * MARGIN_PCT

        # Apply margin and clip to image boundaries
        x1 = int(max(0, x1 - margin_w))
        y1 = int(max(0, y1 - margin_h))
        x2 = int(min(w, x2 + margin_w))
        y2 = int(min(h, y2 + margin_h))

        # Crop, preprocess and stylize
        face_crop = img_pil.crop((x1, y1, x2, y2))
        face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)

        with torch.no_grad():
            fake = model.netG_A(face_tensor)

        # Convert to uint8 RGB image
        fake_np = fake.squeeze().cpu().permute(1, 2, 0).numpy()
        fake_img = ((fake_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)

        # Resize to match original crop and convert to BGR for OpenCV
        w_box, h_box = x2 - x1, y2 - y1
        fake_resized = cv2.resize(fake_img, (w_box, h_box), interpolation=cv2.INTER_LINEAR)
        fake_resized = cv2.cvtColor(fake_resized, cv2.COLOR_RGB2BGR)

        # Paste back into frame
        output[y1:y2, x1:x2] = fake_resized

    # Show result
    cv2.imshow('Human 2 Arcane Live', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions via sys.argv ─────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot', './my_dataset',
    '--name', 'human2arcane',
    '--model', 'cycle_gan',
    '--phase', 'test',
    '--num_test', '1',
    '--load_size', '256',
    '--crop_size', '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing ─────────────────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Video stream ──────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    h, w = frame.shape[:2]

    # ── Stylize entire frame ────────────────────────────────────────────────
    full_tensor = preprocess(img_pil).unsqueeze(0).to(model.device)
    with torch.no_grad():
        stylized_full = model.netG_A(full_tensor)
    stylized_np = stylized_full.squeeze().cpu().permute(1, 2, 0).numpy()
    stylized_bgr = ((stylized_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
    stylized_bgr = cv2.cvtColor(stylized_bgr, cv2.COLOR_RGB2BGR)
    stylized_bgr = cv2.resize(stylized_bgr, (w, h), interpolation=cv2.INTER_LINEAR)

    # ── Face detection ─────────────────────────────────────────────────────
    boxes, _ = mtcnn.detect(img_pil)
    output = stylized_bgr.copy()

    if boxes is not None and len(boxes):
        x1, y1, x2, y2 = [int(v) for v in boxes[0]]
        
        # Apply padding around face (percent-based)
        pad_pct = 0.4  # increase for wider context
        box_w, box_h = x2 - x1, y2 - y1
        pad_w, pad_h = int(pad_pct * box_w), int(pad_pct * box_h)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        # Crop and preprocess face region
        face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
        face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)
        with torch.no_grad():
            stylized_face = model.netG_A(face_tensor)

        face_np = stylized_face.squeeze().cpu().permute(1, 2, 0).numpy()
        face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
        face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)

        # Resize stylized face to original padded region size
        face_bgr_resized = cv2.resize(face_bgr, (x2_pad - x1_pad, y2_pad - y1_pad), interpolation=cv2.INTER_LINEAR)

        # Soft circular mask for blending
        mask = np.zeros((y2_pad - y1_pad, x2_pad - x1_pad, 3), dtype=np.float32)
        cv2.circle(mask, ((x2_pad - x1_pad)//2, (y2_pad - y1_pad)//2), min((x2_pad - x1_pad), (y2_pad - y1_pad))//2, (1,1,1), -1)
        mask = cv2.GaussianBlur(mask, (31, 31), 0)

        # Extract region of interest from background
        roi = output[y1_pad:y2_pad, x1_pad:x2_pad]
        if roi.shape == face_bgr_resized.shape:
            blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
            output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

    # ── Display ─────────────────────────────────────────────────────────────
    cv2.imshow('Human 2 Arcane Composite', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions via sys.argv ─────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot', './my_dataset',
    '--name', 'human2arcane',
    '--model', 'cycle_gan',
    '--phase', 'test',
    '--num_test', '1',
    '--load_size', '256',
    '--crop_size', '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing ─────────────────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Video stream ──────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    h, w = frame.shape[:2]

    # ── Stylize entire frame ────────────────────────────────────────────────
    full_tensor = preprocess(img_pil).unsqueeze(0).to(model.device)
    with torch.no_grad():
        stylized_full = model.netG_A(full_tensor)
    stylized_np = stylized_full.squeeze().cpu().permute(1, 2, 0).numpy()
    stylized_bgr = ((stylized_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
    stylized_bgr = cv2.cvtColor(stylized_bgr, cv2.COLOR_RGB2BGR)
    stylized_bgr = cv2.resize(stylized_bgr, (w, h), interpolation=cv2.INTER_LINEAR)

    # ── Face detection ─────────────────────────────────────────────────────
    boxes, _ = mtcnn.detect(img_pil)
    output = stylized_bgr.copy()

    if boxes is not None and len(boxes):
        x1, y1, x2, y2 = [int(v) for v in boxes[0]]

        # Padding as percentage
        pad_pct = 0.4
        box_w, box_h = x2 - x1, y2 - y1
        pad_w, pad_h = int(pad_pct * box_w), int(pad_pct * box_h)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        # Crop and stylize face
        face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
        face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)
        with torch.no_grad():
            stylized_face = model.netG_A(face_tensor)

        face_np = stylized_face.squeeze().cpu().permute(1, 2, 0).numpy()
        face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
        face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)
        face_bgr_resized = cv2.resize(face_bgr, (x2_pad - x1_pad, y2_pad - y1_pad), interpolation=cv2.INTER_LINEAR)

        # Create a smooth radial blend mask
        h_mask, w_mask = y2_pad - y1_pad, x2_pad - x1_pad
        Y, X = np.ogrid[:h_mask, :w_mask]
        center_y, center_x = h_mask / 2, w_mask / 2
        dist_from_center = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
        max_radius = np.sqrt(center_x**2 + center_y**2)
        radial_mask = 1 - (dist_from_center / max_radius)
        radial_mask = np.clip(radial_mask, 0, 1)
        radial_mask = cv2.GaussianBlur(radial_mask, (81, 81), 0)
        mask = np.dstack([radial_mask] * 3).astype(np.float32)

        # Blend face into stylized background
        roi = output[y1_pad:y2_pad, x1_pad:x2_pad]
        if roi.shape == face_bgr_resized.shape:
            blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
            output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

    # ── Display ─────────────────────────────────────────────────────────────
    cv2.imshow('Human 2 Arcane Composite', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions via sys.argv ─────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot', './my_dataset',
    '--name', 'human2arcane',
    '--model', 'cycle_gan',
    '--phase', 'test',
    '--num_test', '1',
    '--load_size', '256',
    '--crop_size', '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing (no Resize here!) ────────────────────────────────────────
to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
resize_256 = transforms.Resize((256, 256))

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Video stream ──────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    h, w = frame.shape[:2]

    # ── Stylize full frame ──────────────────────────────────────────────────
    full_img_resized = resize_256(img_pil)
    full_tensor = to_tensor(full_img_resized).unsqueeze(0).to(model.device)

    with torch.no_grad():
        stylized_full = model.netG_A(full_tensor)

    stylized_np = stylized_full.squeeze().cpu().permute(1, 2, 0).numpy()
    stylized_bgr = ((stylized_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
    stylized_bgr = cv2.cvtColor(stylized_bgr, cv2.COLOR_RGB2BGR)
    stylized_bgr = cv2.resize(stylized_bgr, (w, h), interpolation=cv2.INTER_LINEAR)

    # ── Face detection ─────────────────────────────────────────────────────
    boxes, _ = mtcnn.detect(img_pil)
    output = stylized_bgr.copy()

    if boxes is not None and len(boxes):
        x1, y1, x2, y2 = [int(v) for v in boxes[0]]

        # Padding as percentage
        pad_pct = 0.4
        box_w, box_h = x2 - x1, y2 - y1
        pad_w, pad_h = int(pad_pct * box_w), int(pad_pct * box_h)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        # ── Crop original resolution, then resize ──────────────────────────
        face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
        face_resized = resize_256(face_crop)
        face_tensor = to_tensor(face_resized).unsqueeze(0).to(model.device)

        with torch.no_grad():
            stylized_face = model.netG_A(face_tensor)

        face_np = stylized_face.squeeze().cpu().permute(1, 2, 0).numpy()
        face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
        face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)

        # Resize stylized face back to match the original padded region
        target_w = x2_pad - x1_pad
        target_h = y2_pad - y1_pad
        face_bgr_resized = cv2.resize(face_bgr, (target_w, target_h), interpolation=cv2.INTER_LINEAR)

        # ── Create radial blend mask ──────────────────────────────────────
        Y, X = np.ogrid[:target_h, :target_w]
        center_y, center_x = target_h / 2, target_w / 2
        dist_from_center = np.sqrt((X - center_x) ** 2 + (Y - center_y) ** 2)
        max_radius = np.sqrt(center_x**2 + center_y**2)
        radial_mask = 1 - (dist_from_center / max_radius)
        radial_mask = np.clip(radial_mask, 0, 1)
        radial_mask = cv2.GaussianBlur(radial_mask, (81, 81), 0)
        mask = np.dstack([radial_mask] * 3).astype(np.float32)

        # ── Blend into stylized background ────────────────────────────────
        roi = output[y1_pad:y2_pad, x1_pad:x2_pad]
        if roi.shape == face_bgr_resized.shape:
            blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
            output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

    # ── Show final composite ───────────────────────────────────────────────
    cv2.imshow('Human 2 Arcane Composite', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions via sys.argv ─────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot', './my_dataset',
    '--name', 'human2arcane',
    '--model', 'cycle_gan',
    '--phase', 'test',
    '--num_test', '1',
    '--load_size', '256',
    '--crop_size', '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing ─────────────────────────────────────────────────────────
to_tensor = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
resize_256 = transforms.Resize((256, 256))

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Video stream ──────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    h, w = frame.shape[:2]

    # ── Stylize full frame ──────────────────────────────────────────────────
    full_img_resized = resize_256(img_pil)
    full_tensor = to_tensor(full_img_resized).unsqueeze(0).to(model.device)

    with torch.no_grad():
        stylized_full = model.netG_A(full_tensor)

    stylized_np = stylized_full.squeeze().cpu().permute(1, 2, 0).numpy()
    stylized_bgr = ((stylized_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
    stylized_bgr = cv2.cvtColor(stylized_bgr, cv2.COLOR_RGB2BGR)
    stylized_bgr = cv2.resize(stylized_bgr, (w, h), interpolation=cv2.INTER_LINEAR)

    # ── Face detection ─────────────────────────────────────────────────────
    boxes, _ = mtcnn.detect(img_pil)
    output = stylized_bgr.copy()

    if boxes is not None and len(boxes):
        x1, y1, x2, y2 = [int(v) for v in boxes[0]]

        # Padding as percentage
        pad_pct = 0.4
        box_w, box_h = x2 - x1, y2 - y1
        pad_w, pad_h = int(pad_pct * box_w), int(pad_pct * box_h)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        # ── Crop & resize ──────────────────────────────────────────────────
        face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
        face_resized = resize_256(face_crop)
        face_tensor = to_tensor(face_resized).unsqueeze(0).to(model.device)

        with torch.no_grad():
            stylized_face = model.netG_A(face_tensor)

        face_np = stylized_face.squeeze().cpu().permute(1, 2, 0).numpy()
        face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
        face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)

        # Resize stylized face back to ROI
        target_w = x2_pad - x1_pad
        target_h = y2_pad - y1_pad
        face_bgr_resized = cv2.resize(face_bgr, (target_w, target_h), interpolation=cv2.INTER_LINEAR)

        # ── Create smoother radial gradient mask using sigmoid ─────────────
        Y, X = np.ogrid[:target_h, :target_w]
        center_y, center_x = target_h / 2, target_w / 2
        dist = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
        max_dist = np.sqrt(center_x**2 + center_y**2)
        norm_dist = dist / max_dist

        # Sigmoid-based smooth mask
        sharpness = 10  # increase for sharper center
        mask = 1 / (1 + np.exp(sharpness * (norm_dist - 0.5)))
        mask = cv2.GaussianBlur(mask, (101, 101), 0)  # soften further
        mask = np.dstack([mask] * 3).astype(np.float32)

        # ── Blend face into background ─────────────────────────────────────
        roi = output[y1_pad:y2_pad, x1_pad:x2_pad]
        if roi.shape == face_bgr_resized.shape:
            blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
            output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

    cv2.imshow('Human 2 Arcane - Smooth Transition', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [5]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions ─────────────────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot',   './my_dataset',
    '--name',       'human2arcane',
    '--model',      'cycle_gan',
    '--phase',      'test',
    '--num_test',   '1',
    '--load_size',  '256',
    '--crop_size',  '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing transform ───────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Start webcam ──────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcnn.detect(img_pil)
    output = frame.copy()

    if boxes is not None and len(boxes):
        h, w = frame.shape[:2]
        box = boxes[0]

        x1, y1, x2, y2 = map(int, box)
        face_w, face_h = x2 - x1, y2 - y1

        # Increase crop region by percentage
        expand_pct = 0.6  # 60% padding
        pad_w = int(face_w * expand_pct / 2)
        pad_h = int(face_h * expand_pct / 2)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        # --- Stylize full frame ---
        full_img = img_pil.resize((256, 256))
        full_tensor = preprocess(full_img).unsqueeze(0).to(model.device)
        with torch.no_grad():
            full_fake = model.netG_A(full_tensor)
        full_np = full_fake.squeeze().cpu().permute(1, 2, 0).numpy()
        full_img_np = ((full_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
        output = cv2.cvtColor(full_img_np, cv2.COLOR_RGB2BGR)
        output = cv2.resize(output, (w, h))

        # --- Stylize cropped face at high resolution ---
        face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
        face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)

        with torch.no_grad():
            face_fake = model.netG_A(face_tensor)

        face_np = face_fake.squeeze().cpu().permute(1, 2, 0).numpy()
        face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
        face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)
        target_w, target_h = x2_pad - x1_pad, y2_pad - y1_pad
        face_bgr_resized = cv2.resize(face_bgr, (target_w, target_h), interpolation=cv2.INTER_LINEAR)

        # --- Gaussian blending mask ---
        mask = np.zeros((target_h, target_w), dtype=np.float32)
        cv2.circle(mask, (target_w // 2, target_h // 2), min(target_w, target_h) // 2, 1.0, -1)
        mask = cv2.GaussianBlur(mask, (61, 61), 0)
        mask = np.expand_dims(mask, axis=2)
        mask = np.repeat(mask, 3, axis=2)

        roi = output[y1_pad:y2_pad, x1_pad:x2_pad]

        # --- Match brightness/color using LAB histogram alignment ---
        if roi.shape == face_bgr_resized.shape:
            face_lab = cv2.cvtColor(face_bgr_resized, cv2.COLOR_BGR2LAB).astype(np.float32)
            roi_lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB).astype(np.float32)

            for c in range(3):
                roi_mean, roi_std = roi_lab[:, :, c].mean(), roi_lab[:, :, c].std()
                face_mean, face_std = face_lab[:, :, c].mean(), face_lab[:, :, c].std()
                face_std = face_std if face_std > 1e-6 else 1.0
                face_lab[:, :, c] = (face_lab[:, :, c] - face_mean) / face_std * roi_std + roi_mean
                face_lab[:, :, c] = np.clip(face_lab[:, :, c], 0, 255)

            face_bgr_resized = cv2.cvtColor(face_lab.astype(np.uint8), cv2.COLOR_LAB2BGR)

        # --- Blend face and background ---
        blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
        output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

    cv2.imshow('Human 2 Arcane Composite', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./my_dataset                  	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [ ]:
#last version but for webcam
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN
import pyvirtualcam

# ── 1) Parse TestOptions ─────────────────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot',   './my_dataset',
    '--name',       'human2arcane',
    '--model',      'cycle_gan',
    '--phase',      'test',
    '--num_test',   '1',
    '--load_size',  '256',
    '--crop_size',  '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing transform ────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Start webcam ───────────────────────────────────────────
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
if not ret:
    raise RuntimeError("Failed to read from webcam.")
h, w = frame.shape[:2]

with pyvirtualcam.Camera(width=w, height=h, fps=30, print_fps=True) as cam:
    print(f'Virtual camera started: {cam.device}')
    print("Press Ctrl+C to quit.")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            boxes, _ = mtcnn.detect(img_pil)
            output = frame.copy()

            if boxes is not None and len(boxes):
                box = boxes[0]
                x1, y1, x2, y2 = map(int, box)
                face_w, face_h = x2 - x1, y2 - y1

                expand_pct = 0.6
                pad_w = int(face_w * expand_pct / 2)
                pad_h = int(face_h * expand_pct / 2)

                x1_pad = max(0, x1 - pad_w)
                y1_pad = max(0, y1 - pad_h)
                x2_pad = min(w, x2 + pad_w)
                y2_pad = min(h, y2 + pad_h)

                full_img = img_pil.resize((256, 256))
                full_tensor = preprocess(full_img).unsqueeze(0).to(model.device)
                with torch.no_grad():
                    full_fake = model.netG_A(full_tensor)
                full_np = full_fake.squeeze().cpu().permute(1, 2, 0).numpy()
                full_img_np = ((full_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
                output = cv2.cvtColor(full_img_np, cv2.COLOR_RGB2BGR)
                output = cv2.resize(output, (w, h))

                face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
                face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)
                with torch.no_grad():
                    face_fake = model.netG_A(face_tensor)
                face_np = face_fake.squeeze().cpu().permute(1, 2, 0).numpy()
                face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
                face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)
                target_w, target_h = x2_pad - x1_pad, y2_pad - y1_pad
                face_bgr_resized = cv2.resize(face_bgr, (target_w, target_h))

                mask = np.zeros((target_h, target_w), dtype=np.float32)
                cv2.circle(mask, (target_w // 2, target_h // 2), min(target_w, target_h) // 2, 1.0, -1)
                mask = cv2.GaussianBlur(mask, (61, 61), 0)
                mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2)

                roi = output[y1_pad:y2_pad, x1_pad:x2_pad]

                if roi.shape == face_bgr_resized.shape:
                    face_lab = cv2.cvtColor(face_bgr_resized, cv2.COLOR_BGR2LAB).astype(np.float32)
                    roi_lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB).astype(np.float32)

                    for c in range(3):
                        roi_mean, roi_std = roi_lab[:, :, c].mean(), roi_lab[:, :, c].std()
                        face_mean, face_std = face_lab[:, :, c].mean(), face_lab[:, :, c].std()
                        face_std = face_std if face_std > 1e-6 else 1.0
                        face_lab[:, :, c] = (face_lab[:, :, c] - face_mean) / face_std * roi_std + roi_mean
                        face_lab[:, :, c] = np.clip(face_lab[:, :, c], 0, 255)

                    face_bgr_resized = cv2.cvtColor(face_lab.astype(np.uint8), cv2.COLOR_LAB2BGR)

                blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
                output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

            # Convert BGR to RGB for virtual cam
            output_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
            cam.send(output_rgb)
            cam.sleep_until_next_frame()

    except KeyboardInterrupt:
        print("Exiting...")

cap.release()


In [ ]:
#for zoom with brightness slider
import sys
import cv2
import torch
import numpy as np
from PIL import Image, ImageEnhance
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN
import pyvirtualcam

# ── 1) Parse TestOptions ─────────────────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot', './my_dataset',
    '--name', 'human2arcane',
    '--model', 'cycle_gan',
    '--phase', 'test',
    '--num_test', '1',
    '--load_size', '256',
    '--crop_size', '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing transform ───────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Webcam init ────────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
if not ret:
    raise RuntimeError("Failed to read from webcam.")
h, w = frame.shape[:2]

# ── 5) Brightness trackbar ────────────────────────────────────────────────────
def nothing(x): pass
cv2.namedWindow("Brightness")
cv2.createTrackbar("Level", "Brightness", 100, 200, nothing)  # 100 is neutral

# ── 6) Start virtual camera ───────────────────────────────────────────────────
with pyvirtualcam.Camera(width=w, height=h, fps=30) as cam:
    print(f"Virtual camera started: {cam.device}")
    print("Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Read trackbar value and apply brightness
        brightness_level = cv2.getTrackbarPos("Level", "Brightness")
        brightness_factor = brightness_level / 100.0  # 1.0 = neutral
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_pil = ImageEnhance.Brightness(img_pil).enhance(brightness_factor)

        boxes, _ = mtcnn.detect(img_pil)
        output = frame.copy()

        if boxes is not None and len(boxes):
            x1, y1, x2, y2 = map(int, boxes[0])
            face_w, face_h = x2 - x1, y2 - y1

            expand_pct = 0.6
            pad_w = int(face_w * expand_pct / 2)
            pad_h = int(face_h * expand_pct / 2)

            x1_pad = max(0, x1 - pad_w)
            y1_pad = max(0, y1 - pad_h)
            x2_pad = min(w, x2 + pad_w)
            y2_pad = min(h, y2 + pad_h)

            # --- Stylize full frame ---
            full_img = img_pil.resize((256, 256))
            full_tensor = preprocess(full_img).unsqueeze(0).to(model.device)
            with torch.no_grad():
                full_fake = model.netG_A(full_tensor)
            full_np = full_fake.squeeze().cpu().permute(1, 2, 0).numpy()
            full_img_np = ((full_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
            output = cv2.cvtColor(full_img_np, cv2.COLOR_RGB2BGR)
            output = cv2.resize(output, (w, h))

            # --- Stylize cropped face at high resolution ---
            face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
            face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)
            with torch.no_grad():
                face_fake = model.netG_A(face_tensor)
            face_np = face_fake.squeeze().cpu().permute(1, 2, 0).numpy()
            face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
            face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)
            target_w, target_h = x2_pad - x1_pad, y2_pad - y1_pad
            face_bgr_resized = cv2.resize(face_bgr, (target_w, target_h))

            # --- Gaussian mask ---
            mask = np.zeros((target_h, target_w), dtype=np.float32)
            cv2.circle(mask, (target_w // 2, target_h // 2), min(target_w, target_h) // 2, 1.0, -1)
            mask = cv2.GaussianBlur(mask, (61, 61), 0)
            mask = np.expand_dims(mask, axis=2)
            mask = np.repeat(mask, 3, axis=2)

            roi = output[y1_pad:y2_pad, x1_pad:x2_pad]
            if roi.shape == face_bgr_resized.shape:
                face_lab = cv2.cvtColor(face_bgr_resized, cv2.COLOR_BGR2LAB).astype(np.float32)
                roi_lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB).astype(np.float32)

                for c in range(3):
                    roi_mean, roi_std = roi_lab[:, :, c].mean(), roi_lab[:, :, c].std()
                    face_mean, face_std = face_lab[:, :, c].mean(), face_lab[:, :, c].std()
                    face_std = face_std if face_std > 1e-6 else 1.0
                    face_lab[:, :, c] = (face_lab[:, :, c] - face_mean) / face_std * roi_std + roi_mean
                    face_lab[:, :, c] = np.clip(face_lab[:, :, c], 0, 255)

                face_bgr_resized = cv2.cvtColor(face_lab.astype(np.uint8), cv2.COLOR_LAB2BGR)

            blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
            output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

        # Show locally
        cv2.imshow("Human 2 Arcane Composite", output)

        # Stream to virtual camera
        cam.send(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
        cam.sleep_until_next_frame()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [4]:
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN
import pyvirtualcam

# ── 1) Parse TestOptions ─────────────────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot',    './my_dataset',
    '--name',        'human2arcane',
    '--model',       'cycle_gan',
    '--phase',       'test',
    '--num_test',    '1',
    '--load_size',   '256',
    '--crop_size',   '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing transform ────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Initialize contrast, gamma, and brightness ───────────────────────────
contrast = 1.0
gamma = 1.0
brightness = 0

def update_contrast(new_contrast):
    global contrast
    contrast = new_contrast / 100.0

def update_gamma(new_gamma):
    global gamma
    gamma = new_gamma / 100.0

def update_brightness(new_brightness):
    global brightness
    brightness = int(new_brightness) - 100

# ── 5) Start webcam and create sliders ───────────────────────────────────────────
cap = cv2.VideoCapture(0)
ret, initial_frame = cap.read()
if not ret:
    raise RuntimeError("Failed to read from webcam.")
h, w = initial_frame.shape[:2]

cv2.namedWindow('Camera Input Adjustments')
cv2.createTrackbar('Contrast', 'Camera Input Adjustments', 100, 200, update_contrast)
cv2.createTrackbar('Gamma', 'Camera Input Adjustments', 100, 200, update_gamma)
cv2.createTrackbar('Brightness', 'Camera Input Adjustments', 100, 200, update_brightness)

with pyvirtualcam.Camera(width=w, height=h, fps=30, print_fps=True) as cam:
    print(f'Virtual camera started: {cam.device}')
    print("Press Ctrl+C to quit.")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Apply contrast, gamma, and brightness adjustments to the input frame
            adjusted_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype(np.float32)
            h_channel, s_channel, v_channel = cv2.split(adjusted_frame)

            # Contrast adjustment on Value channel
            v_channel = np.clip(v_channel * contrast, 0, 255)

            # Gamma adjustment on Value channel
            invGamma = 1.0 / gamma
            v_channel = np.power(v_channel / 255.0, invGamma) * 255.0
            v_channel = np.clip(v_channel + brightness, 0, 255)

            adjusted_frame_hsv = cv2.merge([h_channel, s_channel, v_channel]).astype(np.uint8)
            adjusted_frame_bgr = cv2.cvtColor(adjusted_frame_hsv, cv2.COLOR_HSV2BGR)

            img_pil = Image.fromarray(cv2.cvtColor(adjusted_frame_bgr, cv2.COLOR_BGR2RGB))
            boxes, _ = mtcnn.detect(img_pil)
            output = adjusted_frame_bgr.copy()

            if boxes is not None and len(boxes):
                box = boxes[0]
                x1, y1, x2, y2 = map(int, box)
                face_w, face_h = x2 - x1, y2 - y1

                expand_pct = 0.6
                pad_w = int(face_w * expand_pct / 2)
                pad_h = int(face_h * expand_pct / 2)

                x1_pad = max(0, x1 - pad_w)
                y1_pad = max(0, y1 - pad_h)
                x2_pad = min(w, x2 + pad_w)
                y2_pad = min(h, y2 + pad_h)

                full_img = img_pil.resize((256, 256))
                full_tensor = preprocess(full_img).unsqueeze(0).to(model.device)
                with torch.no_grad():
                    full_fake = model.netG_A(full_tensor)
                full_np = full_fake.squeeze().cpu().permute(1, 2, 0).numpy()
                full_img_np = ((full_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
                output = cv2.cvtColor(full_img_np, cv2.COLOR_RGB2BGR)
                output = cv2.resize(output, (w, h))

                face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
                face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)
                with torch.no_grad():
                    face_fake = model.netG_A(face_tensor)
                face_np = face_fake.squeeze().cpu().permute(1, 2, 0).numpy()
                face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
                face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)
                target_w, target_h = x2_pad - x1_pad, y2_pad - y1_pad
                face_bgr_resized = cv2.resize(face_bgr, (target_w, target_h))

                mask = np.zeros((target_h, target_w), dtype=np.float32)
                cv2.circle(mask, (target_w // 2, target_h // 2), min(target_w, target_h) // 2, 1.0, -1)
                mask = cv2.GaussianBlur(mask, (61, 61), 0)
                mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2)

                roi = output[y1_pad:y2_pad, x1_pad:x2_pad]

                if roi.shape == face_bgr_resized.shape:
                    face_lab = cv2.cvtColor(face_bgr_resized, cv2.COLOR_BGR2LAB).astype(np.float32)
                    roi_lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB).astype(np.float32)

                    for c in range(3):
                        roi_mean, roi_std = roi_lab[:, :, c].mean(), roi_lab[:, :, c].std()
                        face_mean, face_std = face_lab[:, :, c].mean(), face_lab[:, :, c].std()
                        face_std = face_std if face_std > 1e-6 else 1.0
                        face_lab[:, :, c] = (face_lab[:, :, c] - face_mean) / face_std * roi_std + roi_mean
                        face_lab[:, :, c] = np.clip(face_lab[:, :, c], 0, 255)

                    face_bgr_resized = cv2.cvtColor(face_lab.astype(np.uint8), cv2.COLOR_LAB2BGR)

                    blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
                    output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

            # Convert BGR to RGB for virtual cam
            output_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
            cam.send(output_rgb)
            cam.sleep_until_next_frame()

            cv2.imshow('Camera Input Adjustments', adjusted_frame_bgr)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except KeyboardInterrupt:
        print("Exiting...")

cap.release()
cv2.destroyAllWindows()

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./my_dataset                  	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [ ]:
#inference face light

import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions ─────────────────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot',   './my_dataset',
    '--name',       'human2arcane',
    '--model',      'cycle_gan',
    '--phase',      'test',
    '--num_test',   '1',
    '--load_size',  '256',
    '--crop_size',  '256',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing transform ───────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Start webcam ──────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcnn.detect(img_pil)
    output = frame.copy()

    if boxes is not None and len(boxes):
        h, w = frame.shape[:2]
        box = boxes[0]

        x1, y1, x2, y2 = map(int, box)
        face_w, face_h = x2 - x1, y2 - y1

        # Increase crop region by percentage
        expand_pct = 0.6  # 60% padding
        pad_w = int(face_w * expand_pct / 2)
        pad_h = int(face_h * expand_pct / 2)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        # --- Stylize cropped face at high resolution ---
        face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
        face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)

        with torch.no_grad():
            face_fake = model.netG_A(face_tensor)

        face_np = face_fake.squeeze().cpu().permute(1, 2, 0).numpy()
        face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
        face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)
        target_w, target_h = x2_pad - x1_pad, y2_pad - y1_pad
        face_bgr_resized = cv2.resize(face_bgr, (target_w, target_h), interpolation=cv2.INTER_LINEAR)

        # --- Gaussian blending mask ---
        mask = np.zeros((target_h, target_w), dtype=np.float32)
        cv2.circle(mask, (target_w // 2, target_h // 2), min(target_w, target_h) // 2, 1.0, -1)
        mask = cv2.GaussianBlur(mask, (61, 61), 0)
        mask = np.expand_dims(mask, axis=2)
        mask = np.repeat(mask, 3, axis=2)

        roi = output[y1_pad:y2_pad, x1_pad:x2_pad]

        # --- Match brightness/color using LAB histogram alignment ---
        if roi.shape == face_bgr_resized.shape:
            face_lab = cv2.cvtColor(face_bgr_resized, cv2.COLOR_BGR2LAB).astype(np.float32)
            roi_lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB).astype(np.float32)

            for c in range(3):
                roi_mean, roi_std = roi_lab[:, :, c].mean(), roi_lab[:, :, c].std()
                face_mean, face_std = face_lab[:, :, c].mean(), face_lab[:, :, c].std()
                face_std = face_std if face_std > 1e-6 else 1.0
                face_lab[:, :, c] = (face_lab[:, :, c] - face_mean) / face_std * roi_std + roi_mean
                face_lab[:, :, c] = np.clip(face_lab[:, :, c], 0, 255)

            face_bgr_resized = cv2.cvtColor(face_lab.astype(np.uint8), cv2.COLOR_LAB2BGR)

        # --- Blend face and background ---
        blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
        output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

    cv2.imshow('Human 2 Arcane Composite', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
#with proper lighting best results so far are on actuall last epoch for corpped version. makes sense as my datasets are way havier than the example horse
#zebra datasets and they took around 200 epochs to achive known resaults.
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN

# ── 1) Parse TestOptions ─────────────────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot', './my_dataset',
    '--name', 'human2arcane',
    '--model', 'cycle_gan',
    '--phase', 'test',
    '--num_test', '1',
    '--load_size', '256',
    '--crop_size', '256',
    '--epoch', '200',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing transform ───────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Sliders ───────────────────────────────────────────────────────────────
def nothing(x):
    pass

cv2.namedWindow('Controls')
cv2.createTrackbar('Contrast', 'Controls', 100, 300, nothing)   # 1.0–3.0
cv2.createTrackbar('Brightness', 'Controls', 100, 200, nothing) # -1.0–1.0
cv2.createTrackbar('Gamma', 'Controls', 100, 300, nothing)      # 0.1–3.0

# ── 5) Start webcam ──────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcnn.detect(img_pil)
    output = frame.copy()

    # Read slider values
    contrast_val = cv2.getTrackbarPos('Contrast', 'Controls') / 100.0  # 1.0–3.0
    brightness_val = (cv2.getTrackbarPos('Brightness', 'Controls') - 100) / 100.0  # -1.0 to 1.0
    gamma_val = cv2.getTrackbarPos('Gamma', 'Controls') / 100.0  # 0.1–3.0

    if boxes is not None and len(boxes):
        h, w = frame.shape[:2]
        box = boxes[0]
        x1, y1, x2, y2 = map(int, box)
        face_w, face_h = x2 - x1, y2 - y1

        expand_pct = 0.6
        pad_w = int(face_w * expand_pct / 2)
        pad_h = int(face_h * expand_pct / 2)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        # Crop face and apply contrast/brightness/gamma BEFORE inference
        face_crop = np.array(img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))).astype(np.float32) / 255.0

        # Contrast and brightness
        face_crop = np.clip((face_crop - 0.5) * contrast_val + 0.5 + brightness_val, 0, 1)

        # Gamma correction
        face_crop = np.power(face_crop, 1.0 / max(gamma_val, 0.01))

        # Back to PIL for transforms
        face_crop = Image.fromarray((face_crop * 255).astype(np.uint8))

        # Preprocess for model
        face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)
        with torch.no_grad():
            face_fake = model.netG_A(face_tensor)

        face_np = face_fake.squeeze().cpu().permute(1, 2, 0).numpy()
        face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)
        face_bgr = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)

        target_w, target_h = x2_pad - x1_pad, y2_pad - y1_pad
        face_bgr_resized = cv2.resize(face_bgr, (target_w, target_h), interpolation=cv2.INTER_LINEAR)

        # Gaussian blending mask
        mask = np.zeros((target_h, target_w), dtype=np.float32)
        cv2.circle(mask, (target_w // 2, target_h // 2), min(target_w, target_h) // 2, 1.0, -1)
        mask = cv2.GaussianBlur(mask, (61, 61), 0)
        mask = np.expand_dims(mask, axis=2)
        mask = np.repeat(mask, 3, axis=2)

        roi = output[y1_pad:y2_pad, x1_pad:x2_pad]

        # Histogram alignment
        if roi.shape == face_bgr_resized.shape:
            face_lab = cv2.cvtColor(face_bgr_resized, cv2.COLOR_BGR2LAB).astype(np.float32)
            roi_lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB).astype(np.float32)

            for c in range(3):
                roi_mean, roi_std = roi_lab[:, :, c].mean(), roi_lab[:, :, c].std()
                face_mean, face_std = face_lab[:, :, c].mean(), face_lab[:, :, c].std()
                face_std = face_std if face_std > 1e-6 else 1.0
                face_lab[:, :, c] = (face_lab[:, :, c] - face_mean) / face_std * roi_std + roi_mean
                face_lab[:, :, c] = np.clip(face_lab[:, :, c], 0, 255)

            face_bgr_resized = cv2.cvtColor(face_lab.astype(np.uint8), cv2.COLOR_LAB2BGR)

        blended = (mask * face_bgr_resized + (1 - mask) * roi).astype(np.uint8)
        output[y1_pad:y2_pad, x1_pad:x2_pad] = blended

    cv2.imshow('Human 2 Arcane Composite', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./my_dataset                  	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: 200                           	[default: latest]
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0          

In [1]:
## good results on epoch: 40,50 55 with high contrast,90!!!,110 wont look  out of place in the show
#120 125 similar to last epoch less artifacts,130 looks good requires brightness adj,135 prty good some vanishing nose,
#145 big eyes beard still present, 150 best yet, 155 best minor beard artifacts,170 works well no beard artifacts also with glasses
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model

# ── 1) Parse TestOptions via sys.argv ─────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot',   './my_dataset',
    '--name',       'human2arcane',
    '--model',      'cycle_gan',
    '--phase',      'test',
    '--num_test',   '1',
    '--load_size',  '256',
    '--crop_size',  '256',
    '--epoch', '200',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Build full-frame preprocess transform ──────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to model input
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Slider setup ───────────────────────────────────────────────────────────
def nothing(x):
    pass

cv2.namedWindow('Controls')
cv2.createTrackbar('Contrast', 'Controls', 100, 300, nothing)    # 1.0–3.0
cv2.createTrackbar('Brightness', 'Controls', 100, 200, nothing)  # -1.0–1.0
cv2.createTrackbar('Gamma', 'Controls', 100, 300, nothing)       # 0.1–3.0

# ── 4) Start capture ──────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get slider values
    contrast_val = cv2.getTrackbarPos('Contrast', 'Controls') / 100.0
    brightness_val = (cv2.getTrackbarPos('Brightness', 'Controls') - 100) / 100.0
    gamma_val = max(cv2.getTrackbarPos('Gamma', 'Controls') / 100.0, 0.01)

    # Resize input to 256x256 for processing
    resized_input = cv2.resize(frame, (256, 256), interpolation=cv2.INTER_LINEAR)
    input_rgb = cv2.cvtColor(resized_input, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

    # Apply contrast and brightness
    adjusted = np.clip((input_rgb - 0.5) * contrast_val + 0.5 + brightness_val, 0, 1)

    # Apply gamma correction
    adjusted = np.power(adjusted, 1.0 / gamma_val)

    # Convert to PIL for transform
    adjusted_pil = Image.fromarray((adjusted * 255).astype(np.uint8))

    # Preprocess for model
    input_tensor = preprocess(adjusted_pil).unsqueeze(0).to(model.device)

    # Stylize
    with torch.no_grad():
        fake = model.netG_A(input_tensor)

    # Convert back to image
    fake_np = fake.squeeze().cpu().permute(1, 2, 0).numpy()
    fake_img = ((fake_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)

    # Resize back to original frame size
    output = cv2.cvtColor(fake_img, cv2.COLOR_RGB2BGR)
    output = cv2.resize(output, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_LINEAR)

    cv2.imshow('Human 2 Arcane - Full Frame', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./my_dataset                  	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: 200                           	[default: latest]
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0          

In [ ]:
import os
import sys
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN
from IPython.display import display
import matplotlib.pyplot as plt

# ── 1) Parse TestOptions ─────────────────────────────────────────────────────
sys.argv = [
    'test.py',
    '--dataroot', './my_dataset',
    '--name', 'human2arcane',
    '--model', 'cycle_gan',
    '--phase', 'test',
    '--num_test', '50',
    '--load_size', '256',
    '--crop_size', '256',
    '--epoch', '200',
]
opt = TestOptions().parse()
model = create_model(opt)
model.setup(opt)
model.eval()

# ── 2) Preprocessing transform ───────────────────────────────────────────────
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ── 3) Face detector ─────────────────────────────────────────────────────────
mtcnn = MTCNN(keep_all=False)

# ── 4) Inference on image folder ─────────────────────────────────────────────
input_folder = './10imgtest/input/'
output_folder = './10imgtest/output/'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    image_path = os.path.join(input_folder, filename)
    try:
        img_pil_original = Image.open(image_path).convert('RGB')
        img_np_original = np.array(img_pil_original)
    except Exception as e:
        print(f"Error loading original image: {e}")
        continue

    img_pil = img_pil_original.copy() # Create a copy for processing
    boxes, _ = mtcnn.detect(img_pil)
    output = np.array(img_pil)

    if boxes is not None and len(boxes):
        h, w = output.shape[:2]
        box = boxes[0]
        x1, y1, x2, y2 = map(int, box)
        face_w, face_h = x2 - x1, y2 - y1

        expand_pct = 0.4
        pad_w = int(face_w * expand_pct / 2)
        pad_h = int(face_h * expand_pct / 2)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
        face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)

        with torch.no_grad():
            face_fake = model.netG_A(face_tensor)

        face_np = face_fake.squeeze().cpu().permute(1, 2, 0).numpy()
        face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)

        # Resize and blend into original image
        face_bgr_resized = cv2.resize(face_img, (x2_pad - x1_pad, y2_pad - y1_pad))
        output[y1_pad:y2_pad, x1_pad:x2_pad] = face_bgr_resized

    out_path = os.path.join(output_folder, filename)
    cv2.imwrite(out_path, cv2.cvtColor(output, cv2.COLOR_RGB2BGR))
    print(f"Saved translated image: {out_path}")

    # Display images in the notebook
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(img_np_original)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(output)  # Display the RGB 'output' directly
    plt.title("Generated Image")
    plt.axis('off')

    plt.show()

In [ ]:
import os
import sys
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from options.test_options import TestOptions
from models import create_model
from facenet_pytorch import MTCNN
import matplotlib.pyplot as plt
import cv2

def perform_inference_single_epoch(image_path, epoch_to_test=100):
    """
    Performs inference on a single image for a specified epoch.

    Args:
        image_path (str): Path to the input image.
        epoch_to_test (int): The epoch number to use for the checkpoint.
                             Defaults to 100.
    """
    # Base argument list with the specific epoch
    argv = [
        'test.py',
        '--dataroot', './my_dataset',  # Adjust if necessary
        '--name', 'human2arcane',      # Your model name
        '--model', 'cycle_gan',
        '--phase', 'test',
        '--num_test', '1',              # Only testing one image at a time
        '--load_size', '256',
        '--crop_size', '256',
        '--epoch', str(epoch_to_test)
    ]
    sys.argv = argv
    opt = TestOptions().parse()
    opt.num_threads = 0
    opt.batch_size = 1
    opt.serial_batches = True

    # ── 2) Preprocessing transform ───────────────────────────────────────────
    preprocess = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    # ── 3) Face detector ─────────────────────────────────────────────────────
    mtcnn = MTCNN(keep_all=False)

    # Load the single input image
    try:
        img_pil_original = Image.open(image_path).convert('RGB')
        img_np_original = np.array(img_pil_original)
        filename = os.path.basename(image_path)
    except Exception as e:
        print(f"Error loading original image: {e}")
        return

    model = create_model(opt)
    model.setup(opt)
    model.eval()

    img_pil = img_pil_original.copy()
    boxes, _ = mtcnn.detect(img_pil)
    output = np.array(img_pil)

    if boxes is not None and len(boxes):
        h, w = output.shape[:2]
        box = boxes[0]
        x1, y1, x2, y2 = map(int, box)
        face_w, face_h = x2 - x1, y2 - y1

        expand_pct = 0.4
        pad_w = int(face_w * expand_pct / 2)
        pad_h = int(face_h * expand_pct / 2)

        x1_pad = max(0, x1 - pad_w)
        y1_pad = max(0, y1 - pad_h)
        x2_pad = min(w, x2 + pad_w)
        y2_pad = min(h, y2 + pad_h)

        face_crop = img_pil.crop((x1_pad, y1_pad, x2_pad, y2_pad))
        face_tensor = preprocess(face_crop).unsqueeze(0).to(model.device)

        with torch.no_grad():
            face_fake = model.netG_A(face_tensor)

        face_np = face_fake.squeeze().cpu().permute(1, 2, 0).numpy()
        face_img = ((face_np + 1) / 2 * 255).clip(0, 255).astype(np.uint8)

        face_bgr_resized = cv2.resize(face_img, (x2_pad - x1_pad, y2_pad - y1_pad))
        output[y1_pad:y2_pad, x1_pad:x2_pad] = face_bgr_resized

    print("Original Image:")
    plt.figure(figsize=(8, 8))
    plt.imshow(img_np_original)
    plt.axis('off')
    plt.show()
    plt.close()

    print(f"Generated Image (Epoch {epoch_to_test}):")
    plt.figure(figsize=(8, 8))
    plt.imshow(output)
    plt.axis('off')
    plt.show()
    plt.close()



In [ ]:
perform_inference_single_epoch('./10imgtest/input/unnamed.png', epoch_to_test=200)